In [1]:
import pandas as pd

In [2]:
js = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_calendar2018-02-23.json')

js.head()

,calendar,ext_at,property_id,review_count,review_value
0,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:14,9153367,38,4.7
1,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:16,9076897,28,4.6
2,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:19,9123458,71,4.9
3,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:20,9114125,27,4.4
4,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:23,9063683,30,4.9


In [4]:
import datetime
import re
import lxml
from bs4 import BeautifulSoup
import json
#import bs4 as BeautifulSoup
from calendar import monthrange

d = datetime.date.today()
mr = monthrange(d.year, d.month )
cur_month = d.month


fp = open('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar' + datetime.datetime.now().strftime("%Y-%m-%d") + '.json', 'w')


for index, row in js.iterrows():   
    
    all_months = {}
    
    cur_month = d.month

    # Set all the six months
    for i in range(cur_month,cur_month+6):

        mr = monthrange( d.year, i )

        max_day = mr[1]

        all_months[i] = dict.fromkeys(range(1,max_day+1),'')

    
    raw_cal = "<html><body><table>" + row['calendar'] + "</table></body></html>"
    
    soup = BeautifulSoup(raw_cal, 'lxml') # Parse the HTML as a string
    soup_tables = soup.find_all('tbody')# Grab the first table

    for month_tb in soup_tables:
 
        table = soup.find("table")
        
        cells = table.findAll("td")
            
        for td in cells:

            try:
                day_val = int(td.text)

                # If its an unavailable date then mark: 
                m = re.search('c-calendar--unavailable', str(td))

                if m is not None:
                    all_months[cur_month][day_val] = 'UVL' # Unavailable day

                # If its an unavailable date then mark: 
                m = re.search('c-calendar--arrival-day', str(td))

                if m is not None:
                    all_months[cur_month][day_val] = 'ARR' # Arrival day

                # If its an unavailable date then mark: 
                m = re.search('c-calendar--departure-day', str(td))

                if m is not None:
                    all_months[cur_month][day_val] = 'DEP' # Departure day

                m = re.search('c-calendar--available', str(td))
                
                if m is not None:
                    all_months[cur_month][day_val] = 'AVL' # Available day


            except ValueError:
                day_val = None

        # Move to the next month
        cur_month += 1

    # Print the table to check
    #print(all_months)
    
    cal_details = {}
    cal_details['property_id'] = row['property_id']
    cal_details['calendar'] = all_months
    
    # Write the data to JSON
    json.dump(cal_details, fp)
    fp.write('\n,')

    
# Tidy up
fp.close()